In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [14]:
!wget -q https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar -O /tmp/mysql-connector-java-8.0.33.jar

In [ ]:
spark = (
    SparkSession
    .builder
    .appName("Stream From Kafka")
    .config("spark.jars", "/tmp/mysql-connector-java-8.0.33.jar")
    .config("spark.driver.extraClassPath", "/tmp/mysql-connector-java-8.0.33.jar") 
    .config("spark.executor.extraClassPath", "/tmp/mysql-connector-java-8.0.33.jar") 
    .config("spark.streaming.stopGracefullyOnShutdown","true")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0")
    .config("spark.sql.shuffle.partitions",8)
    .master("local[*]")
    .getOrCreate()
)
spark

In [4]:
kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers","ed-kafka:29092")
    .option("subscribe","users")
    .option("startingOffsets","earliest")
    .load()
)
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
#kafka_df.show()

+--------------------+--------------------+-----+---------+------+--------------------+-------------+
|                 key|               value|topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+-----+---------+------+--------------------+-------------+
|[31 37 35 63 34 3...|[7B 22 67 65 6E 6...|users|        6|     0|2025-12-25 15:04:...|            0|
+--------------------+--------------------+-----+---------+------+--------------------+-------------+



In [5]:
kafka_df = kafka_df.withColumns({
    "key":expr("cast(key as string)"),
    "value":expr("cast(value as string)")
})
#kafka_df.show()

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

json_schema = StructType([
    StructField("gender", StringType(), nullable=True),
    StructField("name", StructType([
        StructField("title", StringType(), nullable=True),
        StructField("first", StringType(), nullable=True),
        StructField("last", StringType(), nullable=True)
    ]), nullable=True),
    StructField("location", StructType([
        StructField("street", StructType([
            StructField("number", IntegerType(), nullable=True),
            StructField("name", StringType(), nullable=True)
        ]), nullable=True),
        StructField("city", StringType(), nullable=True),
        StructField("state", StringType(), nullable=True),
        StructField("country", StringType(), nullable=True),
        StructField("postcode", StringType(), nullable=True),  
        StructField("coordinates", StructType([
            StructField("latitude", StringType(), nullable=True),
            StructField("longitude", StringType(), nullable=True)
        ]), nullable=True),
        StructField("timezone", StructType([
            StructField("offset", StringType(), nullable=True),
            StructField("description", StringType(), nullable=True)
        ]), nullable=True)
    ]), nullable=True),
    StructField("email", StringType(), nullable=True),
    StructField("login", StructType([
        StructField("uuid", StringType(), nullable=True),
        StructField("username", StringType(), nullable=True),
        StructField("password", StringType(), nullable=True),
    ]), nullable=True),
    StructField("dob", StructType([
        StructField("date", StringType(), nullable=True),
        StructField("age", IntegerType(), nullable=True)
    ]), nullable=True),
    StructField("registered", StructType([
        StructField("date", StringType(), nullable=True),
        StructField("age", IntegerType(), nullable=True)  
    ]), nullable=True),
    StructField("phone", StringType(), nullable=True),
    StructField("cell", StringType(), nullable=True),
    StructField("id", StructType([
        StructField("name", StringType(), nullable=True),
        StructField("value", StringType(), nullable=True)
    ]), nullable=True),
])

In [7]:
json_kafka = kafka_df.withColumn("json_value",from_json("value",json_schema)).select("json_value.*")
#json_kafka.show()

In [8]:
streaming_data = json_kafka.withColumns({
    "id":regexp_replace(concat(col("id.name"),col("id.value"))," ","-"),
    "first":col("name").first,
    "last":col("name").last,
    "birday":to_date(col("dob").date,"yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"),
    "city":col("location").city,
    "state":col("location").state,
    "country":col("location").country,
    "postcode":col("location").postcode,
    "street":concat(col("location.street.name"),lit("-"),col("location.street.number")),
    "username":col("login").username,
    "password":col("login").password,
    "registered_date":to_date(col("registered.date"),"yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"),
}).drop("name","location","login","registered","dob").select("first","last","username","password","birday","city","state","country","postcode","street","registered_date")

#streaming_data.show()

In [9]:
streaming_data.printSchema()

root
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- username: string (nullable = true)
 |-- password: string (nullable = true)
 |-- birday: date (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- street: string (nullable = true)
 |-- registered_date: date (nullable = true)



In [ ]:
import logging
def save_to_mysql(df,batch_id):
    if df.count()>0:
        logging.info(f"Saving Batch -{batch_id} to DataBase .....")
        try :
            (
                df
                .write 
                .format("jdbc") 
                .option("url", "jdbc:mysql://host.docker.internal:3306/kafka_spark") 
                .option("dbtable", "users") 
                .option("user", "root") 
                .option("password", "YOUR_PASSWORD") 
                .option("driver", "com.mysql.cj.jdbc.Driver") 
                .mode("append") 
                .save()
            )
            logging.info(f"Batch -{batch_id} Seccesfully Saved to DataBase .....")

        except Exception as e:
            logging.error(f"ERROR Saving to DataBase {str(e)}")    

In [ ]:
(
    streaming_data
    .writeStream
    .foreachBatch(save_to_mysql)
    .outputMode("append")
    .trigger(processingTime="5 seconds")
    .option("checkpointLocation","checkpoint-dir-kafka")
    .start()
    .awaitTermination()
)

In [ ]:
(
    streaming_data
    .write 
    .format("jdbc") 
    .option("url", "jdbc:mysql://host.docker.internal:3306/kafka_spark") 
    .option("dbtable", "users") 
    .option("user", "root") 
    .option("password", "YOUR_PASSWORD") 
    .option("driver", "com.mysql.cj.jdbc.Driver") 
    .mode("append") 
    .save()
)